In [9]:
import random
import numpy as np
import pandas as pd
import plotly.express as px

import pypowsybl as pp

pd.options.display.max_columns = None
pd.options.display.expand_frame_repr = False


In [10]:
colors = px.colors.qualitative.Dark24 + px.colors.qualitative.Light24
random.Random(42).shuffle(colors)

# Load a network

You can load your own network with 
net = pp.network.load(...)

In [11]:
def get_upgraded_ieee_net():
    net = pp.network.create_ieee300()

    # add country to substations
    substation_ids = net.get_substations().index
    all_countries = ['AF', 'AX', 'AL', 'DZ', 'AS', 'AD', 'AO', 'AI', 'AQ', 'AG', 'AR', 'AM', 'AW', 'AU', 'AT', 'AZ', 'BS', 'BH', 'BD', 'BB', 'BY', 'BE', 'BZ', 'BJ', 'BM', 'BT', 'BO', 'BQ', 'BA', 'BW', 'BV', 'BR', 'IO', 'BN', 'BG', 'BF', 'BI', 'KH', 'CM', 'CA', 'CV', 'KY', 'CF', 'TD', 'CL', 'CN', 'CX', 'CC', 'CO', 'KM', 'CG', 'CD', 'CK', 'CR', 'CI', 'HR', 'CU', 'CW', 'CY', 'CZ', 'DK', 'DJ', 'DM', 'DO', 'EC', 'EG', 'SV', 'GQ', 'ER', 'EE', 'ET', 'FK', 'FO', 'FJ', 'FI', 'FR', 'GF', 'PF', 'TF', 'GA', 'GM', 'GE', 'DE', 'GH', 'GI', 'GR', 'GL', 'GD', 'GP', 'GU', 'GT', 'GG', 'GN', 'GW', 'GY', 'HT', 'HM', 'VA', 'HN', 'HK', 'HU', 'IS', 'IN', 'ID', 'IR', 'IQ', 'IE', 'IM', 'IL', 'IT', 'JM', 'JP', 'JE', 'JO', 'KZ', 'KE', 'KI', 'KP', 'KR', 'XK', 'KW', 'KG', 'LA', 'LV', 'LB', 'LS', 'LR', 'LY', 'LI', 'LT', 'LU', 'MO', 'MK', 'MG', 'MW', 'MY', 'MV', 'ML', 'MT', 'MH', 'MQ', 'MR', 'MU', 'YT', 'MX', 'FM', 'MD', 'MC', 'MN', 'ME', 'MS', 'MA', 'MZ', 'MM', 'NA', 'NR', 'NP', 'NL', 'NC', 'NZ', 'NI', 'NE', 'NG', 'NU', 'NF', 'MP', 'NO', 'OM', 'PK', 'PW', 'PS', 'PA', 'PG', 'PY', 'PE', 'PH', 'PN', 'PL', 'PT', 'PR', 'QA', 'RE', 'RO', 'RU', 'RW', 'BL', 'SH', 'KN', 'LC', 'MF', 'PM', 'VC', 'WS', 'SM', 'ST', 'SA', 'SN', 'RS', 'SC', 'SL', 'SG', 'SX', 'SK', 'SI', 'SB', 'SO', 'ZA', 'GS', 'SS', 'ES', 'LK', 'SD', 'SR', 'SJ', 'SZ', 'SE', 'CH', 'SY', 'TW', 'TJ', 'TZ', 'TH', 'TL', 'TG', 'TK', 'TO', 'TT', 'TN', 'TR', 'TM', 'TC', 'TV', 'UG', 'UA', 'AE', 'GB', 'US', 'UM', 'UY', 'UZ', 'VU', 'VE', 'VN', 'VG', 'VI', 'WF', 'EH', 'YE', 'ZM', 'ZW']
    number_country = 20
    random.Random(42).shuffle(all_countries)
    substation_countries = random.Random(42).choices(all_countries[:number_country], k=len(substation_ids))
    net.update_substations(id=substation_ids, country=substation_countries)

    # disconnect shunt: not supported yet
    sh = net.get_shunt_compensators().index
    net.update_shunt_compensators(id=sh, connected=[False]*len(sh))

    # generator fix: otherwise they are discarded because of not plausible Pmax
    gen = net.get_generators().index
    net.update_generators(id=gen, max_p=[1000]*len(gen), min_p=[-1000]*len(gen))

    #net.dump("/tmp/test-net.xiidm")

    return net

In [12]:
net = get_upgraded_ieee_net()

# Run a flow decomposition
Add useful columns:
- total loop flow
- total flow
- country terminal 1
- country terminal 2

In [14]:
flow_dec_original=pp.flowdecomposition.run(net)

In [15]:
flow_dec = flow_dec_original.copy()
flow_dec['total_flow'] = flow_dec[[c for c in flow_dec.columns if ('reference' not in c and 'country' not in c)]].sum(axis=1)
flow_dec['total_loop_flow'] = flow_dec[[c for c in flow_dec.columns if 'loop_flow_from_' in c]].sum(axis=1)

countries = set(flow_dec["country1"]).union(set(flow_dec["country2"]))

flow_dec.head()

,commercial_flow,pst_flow,loop_flow_from_ag,loop_flow_from_az,loop_flow_from_be,loop_flow_from_cd,loop_flow_from_cl,loop_flow_from_dz,loop_flow_from_es,loop_flow_from_gr,loop_flow_from_gu,loop_flow_from_hn,loop_flow_from_id,loop_flow_from_iq,loop_flow_from_jo,loop_flow_from_ke,loop_flow_from_kg,loop_flow_from_lv,loop_flow_from_om,loop_flow_from_pn,loop_flow_from_rs,loop_flow_from_rw,ac_reference_flow,dc_reference_flow,country1,country2,total_flow,total_loop_flow
branch_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
L1-5-1,0.0,0.0,3.515342,234.424983,2.886098,5.192526,156.098934,-22.104276,-21.779082,0.415690,6.105495,5.661062,1.436846,1.397081,-0.884369,-2.148660,25.805953,-0.352580,3.799878,0.397073,0.662084,0.685592,NaN,401.182483,AZ,CL,401.215671,401.215671
L100-102-1,0.0,0.0,1.531419,-10.562336,-5.914958,6.148387,5.248153,-7.233396,12.196365,9.223193,0.345833,3.043576,8.329800,7.748080,2.406035,-14.830335,-9.708542,13.581693,12.982713,-7.024001,2.623456,2.245156,NaN,32.200533,AZ,ID,32.380292,32.380292
L102-104-1,0.0,0.0,27.443521,20.055127,-8.794768,-9.189269,-4.117538,20.173960,20.287615,-6.578910,9.020311,-2.100887,-1.308336,2.404690,-12.068429,-15.516320,5.699302,1.914892,4.095777,-3.417192,1.654031,2.906882,NaN,52.162147,ID,GU,52.564459,52.564459
L103-105-1,0.0,0.0,25.594807,16.195352,-11.756622,-19.352100,-4.856852,18.805434,24.458773,-5.291831,3.355045,1.445562,1.725261,4.528265,-10.485583,-3.043226,4.314552,-2.557710,-6.987406,-5.888070,2.290382,3.344206,NaN,35.634149,CD,KE,35.838239,35.838239
L104-108-1,0.0,0.0,27.443521,20.055127,-8.794768,-9.189269,-4.117538,20.173960,20.287615,-8.978910,0.420311,-2.100887,-1.308336,2.404690,-12.068429,-15.516320,5.699302,1.914892,4.095777,-3.417192,1.654031,2.906882,NaN,41.162147,GU,KG,41.564459,41.564459


# Flow decomposition top bar charts

In [16]:
def flow_decomposition_bar_chart(sorting_column, ascending=False, head=20, plot_scatter=True):
    threshold = .05
    df = flow_dec.sort_values(sorting_column, ascending = ascending).head(head).copy()
    df_p = df[[c for c in flow_dec.columns if ('reference' not in c and 'total' not in c and 'country' not in c)]]
    df_m = df_p.abs().div(df_p.abs().sum(axis=1), axis=0) > threshold
    df_f = df_p[df_m]
    df_f['masked_flows_positive'] = df_p[df_p[df_m == False]>0].sum(axis=1)
    df_f['masked_flows_negative'] = df_p[df_p[df_m == False]<0].sum(axis=1)
    fig = px.bar(df_f,
        orientation='h',
        color_discrete_sequence=colors[:df_f.columns.size],
        text_auto='.0f',
        title=f'Sorted by: {sorting_column}, ascending: {ascending}, head: {head}',
        height=1000,
        labels={
            'branch_id':'Branch id',
            'value': 'Flow decomposition value',
            'variable': 'Decomposition part:',
        },
        template="seaborn",
        )
    if plot_scatter:
        fig.add_scatter(
            y=df.index,
            x=df['total_flow'],
            mode='lines+markers',
            name='total_flow'
            )
        fig.add_scatter(
            y=df.index,
            x=df['total_loop_flow'],
            mode='lines+markers',
            name='total_loop_flow'
            )
    fig.show()

In [17]:
flow_decomposition_bar_chart('total_loop_flow')

In [18]:
flow_decomposition_bar_chart('total_loop_flow', ascending=True)

In [19]:
flow_decomposition_bar_chart('total_flow')

In [20]:
flow_decomposition_bar_chart('pst_flow')

In [21]:
flow_decomposition_bar_chart('pst_flow', ascending=True)

In [22]:
flow_decomposition_bar_chart(f'loop_flow_from_{random.Random(42).choice(list(countries)).lower()}')

In [23]:
flow_decomposition_bar_chart(f'loop_flow_from_{random.Random(42).choice(list(countries)).lower()}', ascending=True)

In [24]:
flow_decomposition_bar_chart(f'loop_flow_from_{random.Random(12).choice(list(countries)).lower()}')

# Loop flow repartition from source

In [25]:
c1, c2 = 'country1', 'country2'
df_sum1 = flow_dec.groupby(c1).sum().transpose()
df_sum2 = flow_dec.groupby(c2).sum().transpose()
df_lf_per_country = pd.concat([df_sum1, df_sum2]).groupby(level=0).sum()/2

In [26]:
threshold = .03
df = df_lf_per_country.copy().transpose()
df_p = df[[c for c in df.columns if ('loop_flow_from' in c)]].transpose()
df_s = df_p.abs().sum(axis=1)
df_p = df_p.reindex(df_s.sort_values(ascending=False).index)
df_m = df_p.abs().div(df_p.abs().sum(axis=1), axis=0) > threshold
df_f = df_p[df_m]
df_f['masked_positive'] = df_p[df_p[df_m == False]>0].sum(axis=1)
df_f['masked_negative'] = df_p[df_p[df_m == False]<0].sum(axis=1)
fig = px.bar(df_f,
    orientation='h',
    color_discrete_sequence=colors[:df_f.columns.size],
    text_auto='.0f',
    title='',
    height=1000,
    labels={
        'index':'Origin of loop flow',
        'value': 'Loop flow value per origin',
        'variable': 'Impacted country',
    }
    )
fig.show()

In [27]:
threshold = .03
df = df_lf_per_country.copy().transpose()
df_p = df[[c for c in df.columns if ('loop_flow_from' in c)]].transpose()
df_m = df_p.div(df_p.abs().sum(axis=1), axis=0)
df_s = df_m[df_m < 0].sum(axis=1)
df_m = df_m.reindex(df_s.sort_values(ascending=False).index)
df_n = df_m.abs() > threshold
df_f = df_m[df_n]
df_f['masked_positive'] = df_m[df_p[df_n == False]>0].sum(axis=1)
df_f['masked_negative'] = df_m[df_p[df_n == False]<0].sum(axis=1)
fig = px.bar(df_f,
    orientation='h',
    color_discrete_sequence=colors[:df_f.columns.size],
    text_auto='.2f',
    height=1000,
    labels={
        'index':'Origin of loop flow',
        'value': 'Loop flow value normalized per origin',
        'variable': 'Impacted country',
    }
    )
fig.show()

# Loop flow repartition from destination

In [28]:
threshold = .03
df = df_lf_per_country.copy().transpose()
df_p = df[[c for c in df.columns if ('loop_flow_from' in c)]]
df_s = df_p.abs().sum(axis=1)
df_p = df_p.reindex(df_s.sort_values(ascending=False).index)
df_m = df_p.abs().div(df_p.abs().sum(axis=1), axis=0) > threshold
df_f = df_p[df_m]
df_f['masked_positive'] = df_p[df_p[df_m == False]>0].sum(axis=1)
df_f['masked_negative'] = df_p[df_p[df_m == False]<0].sum(axis=1)
fig = px.bar(df_f,
    orientation='h',
    color_discrete_sequence=colors[:df_f.columns.size],
    text_auto='.0f',
    title='',
    height=1000,
    labels={
        'index':'Destination of loop flow',
        'value': 'Loop flow value per origin',
        'variable': 'Origin country',
    }
    )
fig.show()

In [29]:
threshold = .03
df = df_lf_per_country.copy().transpose()
df_p = df[[c for c in df.columns if ('loop_flow_from' in c)]]
df_m = df_p.div(df_p.abs().sum(axis=1), axis=0)
df_s = df_m[df_m < 0].sum(axis=1)
df_m = df_m.reindex(df_s.sort_values(ascending=False).index)
df_n = df_m.abs() > threshold
df_f = df_m[df_n]
df_f['masked_positive'] = df_m[df_p[df_n == False]>0].sum(axis=1)
df_f['masked_negative'] = df_m[df_p[df_n == False]<0].sum(axis=1)
fig = px.bar(df_f,
    orientation='h',
    color_discrete_sequence=colors[:df_f.columns.size],
    text_auto='.2f',
    height=1000,
    labels={
        'index':'Destination of loop flow',
        'value': 'Loop flow value normalized per origin',
        'variable': 'Origin country',
    }
    )
fig.show()

# Loop flow heat map

In [30]:
df_matrix = df_lf_per_country.transpose().sort_index(axis=1).sort_index()
df_matrix = df_matrix[[c for c in df_matrix.columns if ('loop_flow_from' in c)]].transpose()
df_matrix.index = df_matrix.index.map(lambda c: c.split("_")[-1].upper())
fig = px.imshow(np.sign(df_matrix)*np.log10(df_matrix.abs()+1),
    labels=dict(x="Destination country", y="Source country", color="Loop flow"),
    color_continuous_midpoint=0.0,
    color_continuous_scale="RdBu_r",
    height=800)
fig.update(data=[{'customdata': df_matrix,
    'hovertemplate': 'Destination country: %{x}<br>Source country: %{y}<br>Loop flow: %{customdata}<extra></extra>'}])
int_log_abs_max = int(np.log10(max(df_matrix.max().max(), abs(df_matrix.min().min()))))
tickvals = np.arange(-int_log_abs_max, int_log_abs_max+1)
fig.update_layout(coloraxis_colorbar=dict(
    #title="Population",
    tickvals=tickvals,
    ticktext=[np.sign(v)*10**abs(v) for v in tickvals],
))
fig.show()